In [ ]:
# convert grid into a list of coordinates for each frequency
# we can skip coordinates that have one or less elements
# create a function to check if antennas are in line
# then we can iterate over all pairs of antennas on the grid
# create another function to calculate the distance between antennas that are in line
# we get the coordinates of antinodes based on a formula (make sure to account for boundaries) in a set
# get the intersection of all sets
# then get the count of the number of elements in the final set
import itertools
import math
class Map:
    def __init__(self, grid: list):
        self.grid = grid

    def getAntennaLocations(self):
        grid = self.grid
        locations = dict()
        # iterate over each row then each col of the grid
        for row_idx, row in enumerate (grid):
            for col_idx, char in enumerate(row):
                if char != '.':
                    if char not in locations:
                        locations[char] = [(row_idx, col_idx)]
                    else:
                        locations[char].append((row_idx, col_idx))

        return locations
        
    # get all possible pairs of each antenna
    def getAntennaPairs(self):
        ant_pairs = dict()
        aloc = self.getAntennaLocations()
        for antenna, locations in aloc.items():
            ant_pairs[antenna] = list(itertools.combinations(locations, 2))

        return ant_pairs
    
    # get distance between nodes
    # this also tells us the relative position/directions of these nodes
    def getPairDist(self, ant1: tuple, ant2: tuple):
        (row1, col1) = ant1
        (row2, col2) = ant2
        row_dist = row1-row2
        col_dist = col1-col2
        return row_dist, col_dist
                
        
    # get antinodes
    def getAntinodes(self, pairs: list):
        max_row = len(self.grid)-1
        max_col = len(self.grid[0])-1
        valid_antinodes = []
        antinodes = []
        for pair in pairs:
            (row1, col1) = pair[0]
            (row2, col2) = pair[1]
            row_dist, col_dist = self.getPairDist(pair[0], pair[1])
            # print(pair[0], pair[1], row_dist, col_dist)

            if row_dist < 0 and col_dist < 0:
                # first antenna is higher and to the left of second antenna
                # antinodes will be on left and above first antenna 
                # and on the right and below the second antenna
                antinodes.append((row1-abs(row_dist), col1-abs(col_dist)))
                antinodes.append((row2+abs(row_dist), col2+abs(col_dist)))
            elif row_dist < 0 and col_dist > 0:
                # first antenna is higher and to the right of the second antenna
                # antinodes will be on the right and above the first antenna
                # and on the left and below the second antenna
                antinodes.append((row1-abs(row_dist), col1+abs(col_dist)))
                antinodes.append((row2+abs(row_dist), col2-abs(col_dist)))
            elif row_dist > 0 and col_dist < 0:
                # first antenna is lower and to the left of second antenna
                # antinodes will be on the left and below first antenna 
                # and on the right and above second antenna
                antinodes.append((row1+abs(row_dist), col1-abs(col_dist)))
                antinodes.append((row2-abs(row_dist), col2+abs(col_dist)))
            elif row_dist > 0 and col_dist > 0:
                # first antenna is lower and to the right of second antenna
                # antinodes will be on the right and below first antenna
                # and on the left and above second antenna
                antinodes.append((row1+abs(row_dist), col1+abs(col_dist)))
                antinodes.append((row2-abs(row_dist), col2-abs(col_dist)))
            elif row_dist == 0 and col_dist != 0:
                # antinodes are on the same horizontal plane
                if col_dist > 0:
                    # first antenna is to the right of second
                    antinodes.append((row1, col1+abs(col_dist)))
                    antinodes.append((row2, col2-abs(col_dist)))
                elif col_dist < 0:
                    # first antenna is to the left of second
                    antinodes.append((row1, col1-abs(col_dist)))
                    antinodes.append((row2, col2-abs(col_dist)))
            elif col_dist == 0:
                # antinodes are on the same vertical plane
                if row_dist > 0:
                    # first antenna is lower than second
                    antinodes.append((row1+abs(row_dist), col1))
                    antinodes.append((row2-abs(row_dist), col2))
                elif row_dist < 0:
                    # first antenna is higher than second
                    antinodes.append((row1-abs(row_dist), col1))
                    antinodes.append((row2+abs(row_dist), col2))

    
        # check if coords are valid (not out of bounds)
        for pair in antinodes:
            (row, col) = pair
            if 0 <= row <= max_row and 0 <= col <= max_col:
                valid_antinodes.append(pair)

        return valid_antinodes
    
    def countUniqueAntinodeLocations(self):
        all_locations = []
        for antenna, pairs in self.getAntennaPairs().items():
            all_locations += self.getAntinodes(pairs)
        
        unique_locations = set(all_locations)
        count = len(unique_locations)

        return count, unique_locations
    
    def plotAntinodes(self, unique_locations: set):
        grid = self.grid
        an_grid = [['.' for _ in range(len(grid[0]))] for _ in range(len(grid))]
        # iterate over each row then each col of the grid
        for row_idx, row in enumerate (grid):
            for col_idx, _ in enumerate(row):
                if (row_idx, col_idx) in unique_locations:
                    an_grid[row_idx][col_idx] = "#"

        
        for row in an_grid:
            print(*row, sep=' ')
            print()

In [96]:
with open('data/test/8.txt', 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
map_list = [[char for char in line] for line in lines]
amap = Map(map_list)
# print(amap.getAntennaLocations())
# print(amap.getAntennaPairs())
count, unique_locations = amap.countUniqueAntinodeLocations()
print(count)
# amap.plotAntinodes(unique_locations)

14


In [97]:
with open('data/input/8.txt', 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
map_list = [[char for char in line] for line in lines]
amap = Map(map_list)
count, unique_locations = amap.countUniqueAntinodeLocations()
print(count)

369


In [109]:
# part 2
# we can just add a multiplicator to the function
class Map2(Map):
    def getAntinodes(self, pairs: list):
        max_row = len(self.grid)-1
        max_col = len(self.grid[0])-1
        valid_antinodes = []
        antinodes = []
        for pair in pairs:
            (row1, col1) = pair[0]
            (row2, col2) = pair[1]
            multi = 0
            row_dist, col_dist = self.getPairDist(pair[0], pair[1])
            while min(row1, row2)-abs(row_dist)*multi >= 0 or min(col1, col2)-abs(col_dist)*multi >= 0 or max(row1, row2) + abs(row_dist)*multi <= max_row or max(col1, col2) + abs(col_dist)*multi <= max_col:
                # print(pair[0], pair[1], row_dist, col_dist)
                if row_dist < 0 and col_dist < 0:
                    # first antenna is higher and to the left of second antenna
                    # antinodes will be on left and above first antenna 
                    # and on the right and below the second antenna
                    antinodes.append((row1-abs(row_dist)*multi, col1-abs(col_dist)*multi))
                    antinodes.append((row2+abs(row_dist)*multi, col2+abs(col_dist)*multi))
                elif row_dist < 0 and col_dist > 0:
                    # first antenna is higher and to the right of the second antenna
                    # antinodes will be on the right and above the first antenna
                    # and on the left and below the second antenna
                    antinodes.append((row1-abs(row_dist)*multi, col1+abs(col_dist)*multi))
                    antinodes.append((row2+abs(row_dist)*multi, col2-abs(col_dist)*multi))
                elif row_dist > 0 and col_dist < 0:
                    # first antenna is lower and to the left of second antenna
                    # antinodes will be on the left and below first antenna 
                    # and on the right and above second antenna
                    antinodes.append((row1+abs(row_dist)*multi, col1-abs(col_dist)*multi))
                    antinodes.append((row2-abs(row_dist)*multi, col2+abs(col_dist)*multi))
                elif row_dist > 0 and col_dist > 0:
                    # first antenna is lower and to the right of second antenna
                    # antinodes will be on the right and below first antenna
                    # and on the left and above second antenna
                    antinodes.append((row1+abs(row_dist)*multi, col1+abs(col_dist)*multi))
                    antinodes.append((row2-abs(row_dist)*multi, col2-abs(col_dist)*multi))
                elif row_dist == 0 and col_dist != 0:
                    # antinodes are on the same horizontal plane
                    if col_dist > 0:
                        # first antenna is to the right of second
                        antinodes.append((row1, col1+abs(col_dist)*multi))
                        antinodes.append((row2, col2-abs(col_dist)*multi))
                    elif col_dist < 0:
                        # first antenna is to the left of second
                        antinodes.append((row1, col1-abs(col_dist)*multi))
                        antinodes.append((row2, col2-abs(col_dist)*multi))
                elif col_dist == 0:
                    # antinodes are on the same vertical plane
                    if row_dist > 0:
                        # first antenna is lower than second
                        antinodes.append((row1+abs(row_dist)*multi, col1))
                        antinodes.append((row2-abs(row_dist)*multi, col2))
                    elif row_dist < 0:
                        # first antenna is higher than second
                        antinodes.append((row1-abs(row_dist)*multi, col1))
                        antinodes.append((row2+abs(row_dist)*multi, col2))

                multi += 1
        # check if coords are valid (not out of bounds)
        for pair in antinodes:
            (row, col) = pair
            if 0 <= row <= max_row and 0 <= col <= max_col:
                valid_antinodes.append(pair)

        return valid_antinodes

In [113]:
with open('data/input/8.txt', 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
map_list = [[char for char in line] for line in lines]
amap = Map2(map_list)
# print(amap.getAntennaLocations())
# print(amap.getAntennaPairs())
count, unique_locations = amap.countUniqueAntinodeLocations()
print(count)
# amap.plotAntinodes(unique_locations)

1169
